In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pylab as plt
import nltk
import glob

In [3]:
ham_address = '/Volumes/Extra/Columbia/Fall2018/Classes/ML/enron1/ham/'
spam_address = '/Volumes/Extra/Columbia/Fall2018/Classes/ML/enron1/spam/'

In [4]:
summary_file = open('/Volumes/Extra/Columbia/Fall2018/Classes/ML/enron1/Summary.txt')
summary = summary_file.read()
summary_file.close()

In [5]:
hams = []
hamfiles = sorted(glob.glob(ham_address+'*.txt'))
hfiles = len(hamfiles)
for i in range(hfiles):
    file = open(hamfiles[i], 'rt')
    text = file.read()
    hams.append(text)
    file.close()

In [6]:
spams = []
spamfiles = sorted(glob.glob(spam_address+'*.txt'))
sfiles = len(spamfiles)
for i in range(sfiles):
    file = open(spamfiles[i], 'rt', encoding="latin-1")
    text = file.read()
    spams.append(text)
    file.close()

In [7]:
## word stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
ps = PorterStemmer()
#nltk.download()

In [197]:
phrase = ['I likes you like you tree']

In [202]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
words=tokenizer.tokenize(phrase[0])
emp = []
for w in words:
    emp.append(ps.stem(w))

In [213]:
emp

['I', 'like', 'you', 'like', 'you', 'tree']

In [214]:
def embed_one(datalist):
    """
    Construct stemmed+bag-of-words model for individual then construct an array of individual bags
    
    Returns
    a collection of individual set dict corresponding to its counts
    """
    bag_collection = []
    ndata = len(datalist)
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    
    for i in range(ndata):
        stemmed = []
        token = tokenizer.tokenize(datalist[i])
        for w in token:
            stemmed.append(ps.stem(w))
    
        stemmed = list(set(stemmed))
        nstem = len(stemmed)
        one_bag = {}
        
        for j in range(nstem):
            key = stemmed[j]
            if key in one_bag:
                one_bag[key] += 1
            else:
                one_bag[key] = 1
        bag_collection.append(one_bag)
        
    return bag_collection

def embed_whole(datalist):
    """
    From a list of data (should have multiple), do stemming (+remove non-words) then apply the bag-of-words model
    
    Returns
    a dictionary of bag-of-words each dic corresponding to its counts
    """
    bag = {}
    ndata = len(datalist)
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    
    for i in range(ndata):
        stemmed = []
        token = tokenizer.tokenize(datalist[i])
        for w in token:
            stemmed.append(ps.stem(w))
    
        stemmed = list(set(stemmed))
        nstem = len(stemmed)
        for j in range(nstem):
            key = stemmed[j]
            if key in bag:
                bag[key] += 1
            else:
                bag[key] = 1        
    return bag
    

In [215]:
ham_one_bag = embed_one(hams)
spam_one_bag = embed_one(spams)

In [216]:
whole = np.concatenate((hams, spams))
whole_bag = embed_data(whole)

In [217]:
spam_sum_bag = embed_whole(spams)
ham_sum_bag = embed_whole(hams)

In [218]:
spam_sum_bag

{'cholesterol': 18,
 'alert': 36,
 'begin': 44,
 'resist': 6,
 'in': 618,
 'potenc': 3,
 'least': 17,
 '90': 66,
 'but': 203,
 'restor': 8,
 'wound': 11,
 'heal': 5,
 '95': 56,
 'sexual': 45,
 'by': 350,
 'are': 495,
 'hair': 14,
 'skin': 20,
 'we': 555,
 'densiti': 2,
 'our': 452,
 'up': 257,
 'less': 94,
 'refer': 47,
 'is': 671,
 'gone': 20,
 'strength': 8,
 'nearli': 16,
 'ha': 227,
 'as': 396,
 'read': 83,
 'introduc': 22,
 'level': 39,
 'hgh': 5,
 'subject': 1500,
 'reduc': 34,
 'improv': 50,
 'forti': 7,
 'stabil': 6,
 'young': 32,
 'lower': 30,
 'doctor': 89,
 'near': 36,
 'everyon': 37,
 'heart': 25,
 'scienc': 12,
 'sleep': 37,
 'common': 21,
 'it': 542,
 'swing': 5,
 'growth': 54,
 'produc': 41,
 'human': 12,
 'increas': 105,
 'mood': 3,
 'dobmeo': 1,
 'master': 18,
 'advantag': 33,
 'thi': 563,
 'pressur': 8,
 'hormon': 5,
 'and': 866,
 'at': 415,
 'with': 534,
 'formul': 6,
 'diminish': 1,
 'cellulit': 3,
 'normal': 44,
 'wrinkl': 9,
 'muscl': 31,
 'textur': 1,
 'unsubscri

In [13]:
def overlap(dict1, dict2):
    """
    Find the overlapping dictionaries
    """
    key1 = set(dict1.keys())
    key2 = set(dict2.keys())
    intersection = key1 & key2
    newdict1 = {}
    newdict2 = {}
    intersection = list(intersection)
    ninter = len(intersection)
    for i in range(ninter):
        newdict1[intersection[i]] = dict1[intersection[i]]
        newdict2[intersection[i]] = dict2[intersection[i]]
    return newdict1, newdict2
    

In [230]:
ovlp_spam, ovlp_ham = overlap(spam_bag, ham_bag)

In [231]:
a= ovlp_spam.keys()

In [177]:
def Naive_Bayes(test, train):
    """
    Compute the score of test with respect to training sets using Naive Bayes
    
    Returns to scores
    """
    
    
    

def NN_Distances(test, train, nn_option='L2'):
    """
    Compute the distance of test based on training sets using Nearest Neighbor
    test: test [one_email] ; one(training set 1 [list]); two(training set 2 [list])
    
    Returns to the array of distances
    """
    testing = embed_one(test)[0] ## because it's just one
    testkeys = testing.keys()
    ntest = len(testkeys)

    ntrain = len(train)
    dist = np.zeros(ntrain)
    
    for i in range(ntrain):
        onekey = train[i].keys()
        one_eval = testing.copy()
        one_eval.update(train[i]) ## for all un-matching dictionaries
        for j in range(ntest):
            if list(testkeys)[j] in onekey:
                thiskey = list(testkeys)[j]
                one_eval[thiskey] = train[i][thiskey] - testing[thiskey] ## subtract only when items are matching
            else:
                pass
            
        one_eval = np.array(list(one_eval.values()))
        print (one_eval)
        if option == 'L1' or option == 'Linf':
            dist[i] = sum(np.abs(one_eval))
        elif option == 'L2':
            dist[i] = np.sqrt(sum(one_eval**2))
            
    return dist


In [ ]:
## Naive Bayes, Decision Tree, NN Classifiers
def Classifier(new, one, two, option='NB', nn_option='L2'):
    """
    For "overlapping" bag of words between new and training sets, evaluate probability based on classifier of choice
    NB: Naive Bayes; DT: Decision Tree; NN: Nearest Neighbors
    New (to-be-examined); one(training 1); two(training 2)
    """
    
    common_new, common_one = overlap(new, one)
    
    ckeys = common
    ncommon = len(common)
    
    scores = np.zeros(2)
    if option == 'NN':
        dist_one = NN_Distances(new, one, nn_option)
        dist_two = NN_Distances(new, two, nn_option)
        if nn_option == 'L1' or nn_option == 'L2':
            scores[0] = np.sum(dist_one)
            scores[1] = np.sum(dist_two)
        elif nn_option == 'Linf':
            scores[0] = np.max(dist_one)
            scores[1] = np.max(dist_two)
    
    elif option == 'NB':
        
            
    elif option == 'DT':
            
    return scores

In [47]:
### only evaluate the overlapping keywords?
inter_spam, inter_ham = overlap(spam_bag, ham_bag)

In [233]:
### INCORRECT (Without Stemming)
def embed_whole(datalist):
    """
    From a list of data (should have multiple), do stemming (+remove non-words) then apply the bag-of-words model
    
    Returns
    a dictionary of bag-of-words each dic corresponding to its counts
    """
    bag = {}
    ndata = len(datalist)
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    
    for i in range(ndata):
        stemmed = tokenizer.tokenize(datalist[i])
        nstem = len(stemmed)
        for j in range(nstem):
            key = stemmed[j]
            if key in bag:
                bag[key] += 1
            else:
                bag[key] = 1        
    return bag

def embed_one(datalist):
    """
    Construct stemmed+bag-of-words model for individual then construct an array of individual bags
    
    Returns
    a collection of individual set dict corresponding to its counts
    """
    bag_collection = []
    ndata = len(datalist)
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    
    for i in range(ndata):
        one_bag = {}
        stemmed = tokenizer.tokenize(datalist[i])
        nstem = len(stemmed)
        for j in range(nstem):
            key = stemmed[j]
            if key in one_bag:
                one_bag[key] += 1
            else:
                one_bag[key] = 1
        bag_collection.append(one_bag)
        
    return bag_collection